In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]='false'

In [2]:
from pathlib import Path
import numpy as np

from flowrec.training_and_states import TrainingState

In [3]:
import jax
import jax.numpy as jnp
import haiku as hk

In [38]:
from flowrec.models.feedforward import Model as FeedForward
rng = jax.random.PRNGKey(5)
x = jnp.array([1,2,3]).astype('float32')
y = jnp.array([1,2,3,4,5]).astype('float32')
mlp = FeedForward([2,5],rng)
params = mlp.init(x)

In [39]:
# print(params)
for key,val in params.items():
    print(key)
    print(val)
    print(type(key),type(val))

mlp/~/linear_0
{'b': DeviceArray([0., 0.], dtype=float32), 'w': DeviceArray([[ 1.0389447 ,  0.90172136],
             [-0.03673628,  0.2453191 ],
             [-1.0663218 , -0.9265131 ]], dtype=float32)}
<class 'str'> <class 'dict'>
mlp/~/linear_1
{'b': DeviceArray([0., 0., 0., 0., 0.], dtype=float32), 'w': DeviceArray([[-0.73909765,  0.00764927, -0.5126396 ,  0.16823475,
              -0.24050795],
             [ 0.3720918 ,  0.8736663 ,  0.27848655,  0.24627039,
               0.91382617]], dtype=float32)}
<class 'str'> <class 'dict'>


In [40]:
norm = 0.
for layer,weights in params.items():
    for name,w in weights.items():
        norm += regulariser_l2(w)
        print(f'Layer {layer}:{name} has {w.shape} weights. Sum of square {regulariser_l2(w)}')
print(norm)

Layer mlp/~/linear_0:b has (2,) weights. Sum of square 0.0
Layer mlp/~/linear_0:w has (3, 2) weights. Sum of square 3.949507236480713
Layer mlp/~/linear_1:b has (5,) weights. Sum of square 0.0
Layer mlp/~/linear_1:w has (2, 5) weights. Sum of square 2.7702972888946533
6.719805


In [41]:
def regulariser_l2(weights):
    f_norm = jnp.sum(jnp.square(weights))
    return f_norm

In [42]:
b = jax.tree_util.tree_leaves(params)

In [46]:
a = jax.tree_util.tree_map(regulariser_l2,params)

In [62]:
leaves = jax.tree_util.tree_leaves(a)
np.sum(leaves)

6.719805

In [57]:
c, c_structure = jax.tree_util.tree_flatten(params)
jax.tree_util.tree_unflatten(c_structure, c)

{'mlp/~/linear_0': {'b': DeviceArray([0., 0.], dtype=float32),
  'w': DeviceArray([[ 1.0389447 ,  0.90172136],
               [-0.03673628,  0.2453191 ],
               [-1.0663218 , -0.9265131 ]], dtype=float32)},
 'mlp/~/linear_1': {'b': DeviceArray([0., 0., 0., 0., 0.], dtype=float32),
  'w': DeviceArray([[-0.73909765,  0.00764927, -0.5126396 ,  0.16823475,
                -0.24050795],
               [ 0.3720918 ,  0.8736663 ,  0.27848655,  0.24627039,
                 0.91382617]], dtype=float32)}}